# Prepare to use Apache Spark
**Apache Spark:** is a **distributed data processing framework** that enables large-scale data analytics by coordinating work across multiple processing nodes in a cluster. Put more simply, Spark uses a "divide and conquer" approach to processing large volumes of data quickly by distributing the work across **multiple computers**.

Spark can run code written in a **wide range of languages**, including Java, Scala (a Java-based scripting language), Spark R, Spark SQL, and PySpark (a Spark-specific variant of Python).

## Spark settings
In Microsoft Fabric, each workspace is assigned a **Spark cluster**. An administrator can manage settings for the Spark cluster in the Data Engineering/Science section of the workspace settings.

<img src="./images/08/spark-settings.png" alt="Spark settings" style="border: 2px solid black; border-radius: 10px;">

Specific configuration settings include:
- **Node Family:** The type of **virtual machines** used for the Spark cluster nodes. In most cases, **memory optimized** nodes provide optimal performance.
- **Runtime version:** The **version** of Spark (and dependent subcomponents) to be run on the cluster.
- **Spark Properties:** Spark-specific **settings** that you want to enable or override in your cluster. You can see a list of properties in the [Apache Spark documentation](https://spark.apache.org/docs/latest/configuration.html#available-properties).

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> In most scenarios, the default settings provide an optimal configuration for Spark in Microsoft Fabric.

## Libraries
**The Spark open source ecosystem** includes a wide selection of **code libraries** for common (and sometimes very specialized) tasks. Since a great deal of Spark processing is performed using PySpark, the huge range of Python libraries ensures that whatever the task you need to perform, there's probably a library to help.

By default, Spark clusters in Microsoft Fabric include many of the **most commonly used libraries**. In order to set additional default libraries or persist library specifications for code items, you need **`workspace admin permissions`** to create an environment and set the default environment for the workspace.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> For more information about library management, see Manage Apache Spark libraries in [Microsoft Fabric in the Microsoft Fabric documentation](https://learn.microsoft.com/en-us/fabric/data-engineering/library-management).

# Run Spark code
To edit and run Spark code in Microsoft Fabric, you can use **`notebooks`**, or you can define a **`Spark job`**.

## Notebooks
**`Notebooks`** enable you to combine text, images, and code written in **multiple languages** to create an interactive item that you can share with others and collaborate.
- Notebooks consist of one or more cells, each of which can contain markdown-formatted content or executable code. You can run the code interactively in the notebook and see the results immediately.

<img src="./images/08/notebook.png" alt="Notebook in Microsoft Fabric" style="border: 2px solid black; border-radius: 10px;">

## Spark job definition
If you want to use Spark to **ingest** and **transform** data as part of an automated process, you can define a **`Spark job`** to run a script on-demand or based on a schedule.
- To configure a Spark job, create a *`Spark Job Definition`** in your workspace and specify the script it should run. 
- You can also specify a reference file (for example, a Python code file containing definitions of functions that are used in your script) and a reference to a specific lakehouse containing data that the script processes.

<img src="./images/08/spark-job.png" alt="Spark job definition in Microsoft Fabric" style="border: 2px solid black; border-radius: 10px;">

# Work with data in a Spark dataframe
Natively, Spark uses a data structure called a **`resilient distributed dataset (RDD)`**; but while you can write code that works directly with RDDs, the most commonly used data structure for working with structured data in Spark is the **`dataframe`**, which is provided as part of the Spark SQL library. 
- Dataframes in Spark are similar to those in the ubiquitous Pandas Python library, but optimized to work in Spark's distributed processing environment.

## Loading data into a dataframe
Sample CSV file:

```
ProductID,ProductName,Category,ListPrice
771,"Mountain-100 Silver, 38",Mountain Bikes,3399.9900
772,"Mountain-100 Silver, 42",Mountain Bikes,3399.9900
773,"Mountain-100 Silver, 44",Mountain Bikes,3399.9900
...
```

### Inferring a schema
In a Spark notebook, you could use the following PySpark code to load the file data into a dataframe and display the first 10 rows:

In [0]:
%%pyspark
df = spark.read.load('Files/data/products.csv',
    format='csv',
    header=True
)
display(df.limit(10))

In [0]:
%%spark
val df = spark.read.format("csv").option("header", "true").load("Files/data/products.csv")
display(df.limit(10))

The both provide same output:

<img src="./images/08/output_inferring_schema.png" alt="Output Inferring Schema" style="border: 2px solid black; border-radius: 10px;">

### Specifying an explicit schema
You can also specify an explicit schema for the data, which is useful when the column names aren't included in the data file, like this CSV example:
```
771,"Mountain-100 Silver, 38",Mountain Bikes,3399.9900
772,"Mountain-100 Silver, 42",Mountain Bikes,3399.9900
773,"Mountain-100 Silver, 44",Mountain Bikes,3399.9900
...
```

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

productSchema = StructType([
    StructField("ProductID", IntegerType()),
    StructField("ProductName", StringType()),
    StructField("Category", StringType()),
    StructField("ListPrice", FloatType())
    ])

df = spark.read.load('Files/data/product-data.csv',
    format='csv',
    schema=productSchema,
    header=False)
display(df.limit(10))

The result would be the same

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> Specifying an explicit schema also **`improves performance`**!

## Filtering and grouping dataframes


In [0]:
## For example, the following code example uses the select method to retrieve the ProductID and ListPrice columns from the df dataframe containing product data in the previous example:
pricelist_df = df.select("ProductID", "ListPrice")

#Selecting a subset of columns from a dataframe is a common operation, which can also be achieved by using the following shorter syntax:
pricelist_df = df["ProductID", "ListPrice"]

In [0]:
#For example, this example code chains the select and where methods to create a new dataframe containing the ProductName and ListPrice columns for products with a category of Mountain Bikes or Road Bikes:
bikes_df = df.select("ProductName", "Category", "ListPrice").where((df["Category"]=="Mountain Bikes") | (df["Category"]=="Road Bikes"))
display(bikes_df)

In [0]:
#For example, the following PySpark code counts the number of products for each category:
counts_df = df.select("ProductID", "Category").groupBy("Category").count()
display(counts_df)

## Saving a dataframe

In [0]:
# The following code example saves the dataFrame into a parquet file in the data lake, replacing any existing file of the same name.
bikes_df.write.mode("overwrite").parquet('Files/product_data/bikes.parquet')

## Partitioning the output file
**Partitioning:** is an optimization technique that enables Spark to **maximize performance** across the worker nodes. More performance gains can be achieved when filtering data in queries by eliminating unnecessary disk IO.

To save a dataframe as a partitioned set of files, use the **`partitionBy`** method when writing the data. 

In [0]:
#The following example saves the bikes_df dataframe (which contains the product data for the mountain bikes and road bikes categories), and partitions the data by category:
bikes_df.write.partitionBy("Category").mode("overwrite").parquet("Files/bike_data")

The folder names generated when partitioning a dataframe include the partitioning column name and value in a **`column=value format`**, so the code example creates a folder named **`bike_data`** that contains the following **subfolders**:
- Category=Mountain Bikes
- Category=Road Bikes


<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> You can partition the data by **`multiple columns`**, which results in a **`hierarchy of folders`** for each partitioning key. For example, you might partition sales order data by year and month, so that the folder hierarchy includes a folder for each year value, which in turn contains a subfolder for each month value.

## Load partitioned data
When reading partitioned data into a dataframe, you can load data from any folder within the hierarchy by specifying explicit values or wildcards for the partitioned fields.

In [0]:
# When reading partitioned data into a dataframe, you can load data from any folder within the hierarchy by specifying explicit values or wildcards for the partitioned fields. The following example loads data for products in the Road Bikes category:
road_bikes_df = spark.read.parquet('Files/bike_data/Category=Road Bikes')
display(road_bikes_df.limit(5))

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> The **partitioning columns** specified in the file path are **`omitted`** in the resulting dataframe. The results produced by the example query would not include a Category column - the category for all rows would be Road Bikes.

# Work with data using Spark SQL
**`Spark SQL`** enables data analysts to use **SQL expressions** to query and manipulate data.

## Creating database objects in the Spark catalog
**The Spark catalog:** is a **`metastore`** for **relational data objects** such as views and tables. 
- **The Spark runtime** can use the **catalog** to seamlessly integrate code written in any Spark-supported language with SQL expressions that may be more natural to some data analysts or developers.

### Temporary View
One of the simplest ways to make data in a dataframe available for querying in the Spark catalog is to create a **`temporary view`**
- A view is **temporary**, meaning that it's automatically deleted at the end of the current session, as shown in the following code example:

In [0]:
df.createOrReplaceTempView("products_view")

### Tables
**`Tables`** are **metadata structures** that store their underlying data in the **storage location** associated with the catalog.
- Tables are **persisted in the catalog** to define a database that can be queried using Spark SQL.
- In Microsoft Fabric, data for managed tables is stored in the Tables storage location shown in **`data lake`**, and any tables created using Spark are listed there.

You can create an empty table by using the `spark.catalog.createTable` method, or you can save a dataframe as a table by using its `saveAsTable` method. Deleting a managed table also deletes its underlying data.

For example, the following code saves a dataframe as a new table named products:

In [0]:
df.write.format("delta").saveAsTable("products")

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> The Spark catalog supports tables based on files in various formats. The preferred format in Microsoft Fabric is **`delta`**, which is the format for a relational data technology on Spark named Delta Lake. Delta tables support features commonly found in relational database systems, including transactions, versioning, and support for streaming data.

Additionally, you can create external tables by using the `spark.catalog.createExternalTable` method. External tables define metadata in the catalog but get their underlying data from an **external storage location**; typically a folder in the Files storage area of a lakehouse. Deleting an external table doesn't delete the underlying data.

## Using the Spark SQL API to query data
You can use the **`Spark SQL API`** in code written in any language to query data in the catalog. 

For example, the following PySpark code uses a SQL query to return data from the products table as a dataframe.

In [0]:
bikes_df = spark.sql("SELECT ProductID, ProductName, ListPrice \
                      FROM products \
                      WHERE Category IN ('Mountain Bikes', 'Road Bikes')")
display(bikes_df)

## Using SQL code
In a notebook, you can also use the %%sql magic to run **`SQL code`** that queries objects in the catalog, like this:

In [0]:
%%sql

SELECT Category, COUNT(ProductID) AS ProductCount
FROM products
GROUP BY Category
ORDER BY Category

# Visualize data in a Spark notebook

## Using built-in notebook charts
By default, results are rendered as a table, but you can also change the results view to a chart and use the **`chart properties`** to **customize** how the chart visualizes the data, as shown here:

<img src="./images/08/notebook-chart.png" alt="Notebook chart" style="border: 2px solid black; border-radius: 10px;">

## Using graphics packages in code
There are many **graphics packages** that you can use to create data visualizations in code. In particular, Python supports a large selection of packages; most of them built on the base Matplotlib library. 
- The output from a graphics library can be rendered in a notebook, making it easy to combine code to ingest and manipulate data with inline data visualizations and markdown cells to provide commentary.

For example, you could use the following PySpark code to aggregate data from the hypothetical products data explored previously in this module, and use Matplotlib to create a chart from the aggregated data.

In [0]:
from matplotlib import pyplot as plt

# Get the data as a Pandas dataframe
data = spark.sql("SELECT Category, COUNT(ProductID) AS ProductCount \
                  FROM products \
                  GROUP BY Category \
                  ORDER BY Category").toPandas()

# Clear the plot area
plt.clf()

# Create a Figure
fig = plt.figure(figsize=(12,8))

# Create a bar plot of product counts by category
plt.bar(x=data['Category'], height=data['ProductCount'], color='orange')

# Customize the chart
plt.title('Product Counts by Category')
plt.xlabel('Category')
plt.ylabel('Products')
plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)
plt.xticks(rotation=70)

# Show the plot area
plt.show()

The Matplotlib library requires data to be in a Pandas dataframe rather than a Spark dataframe, so the toPandas method is used to convert it. The code then creates a figure with a specified size and plots a bar chart with some custom property configuration before showing the resulting plot.

The chart produced by the code would look similar to the following image:

<img src="./images/08/chart.png" alt="Chart" style="border: 2px solid black; border-radius: 10px;">